# Libraries

## Agentic Related

In [17]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from openai import AsyncOpenAI
from agents.model_settings import ModelSettings

## Exterior

In [12]:
from pydantic import BaseModel
import asyncio
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

## Interior

In [13]:
from dotenv import load_dotenv
import os
from typing import Dict
from IPython.display import display, Markdown

# Development

## Loading in Gemini and OpenAI Models

In [30]:
# OpenAI
load_dotenv(override=True)
gpt_model = "gpt-4o-mini"

# Gemini
gemini = {
    "base_model": "gemini-2.0-flash",
    "api_key": os.getenv("GEMINI_API_KEY"),
    "base_url": "https://generativelanguage.googleapis.com/v1beta"
}

gemini_client = AsyncOpenAI(base_url=gemini["base_url"], api_key=gemini["api_key"])
gemini_model = OpenAIChatCompletionsModel(model=gemini["base_model"], openai_client=gemini_client)

## Development Summary
* OpenAI Agents SDK includes some hosted tools, these include:\
WebSearchTool --> Allows an agent to search the web.\
FileSearchTool --> Allows retrieving information from OpenAI Vector Stores.\
ComputerTool --> Allows automating computer use tasks. These could include taking screenshots, clicking, searching, or other operations, which would require guardrails to limit certain conditions.

## Search Agent

In [36]:
# The search agent's instructions parameters values. (these can be changed based on your liking)
search_agent_instructions_params = {
    "min_paragraph_count": 2,
    "max_paragraph_count": 3,
    "min_word_count": 200,
    "max_word_count": 300,
    "tone": "semi-formal"
}

In [37]:
# Search's dedicated instructions
search_agent_instructions = f" You are a research assistant.\
                Given a search term, you search the web for that term and produce a concise summary of the results.\
                The summary must be {search_agent_instructions_params["min_paragraph_count"]}-{search_agent_instructions_params["max_paragraph_count"]} paragraphs and between than {search_agent_instructions_params["min_word_count"]}-{search_agent_instructions_params["max_word_count"]} words.\
                Capture the main points. Write in a f{search_agent_instructions_params["tone"]} tone.\
                This will be consumed by someone synthesizig a report, so it's vital you capture the essense and ignore any fluff.\
                Do not include any additional commentary other than the summary itself."

In [40]:
search_agent = Agent(
    name="Search Agent",
    instructions=search_agent_instructions,
    # due to the costs of using the websearchtool, i will use the low settings.
    tools=[WebSearchTool(search_context_size="low")],
    model=gpt_model,
    # search agent is expected to run the tool. 
    model_settings=ModelSettings(tool_choice="required")
)

In [ ]:
sample_agent_prompt = "Apple New Products 2025"

with trace("Search"):
    result = await Runner.run(search_agent, sample_agent_prompt)
display(Markdown(result.final_output))